In [16]:
import time
import numpy as np
import MatchSummarizer as MS
import importlib 
importlib.reload(MS)
MatchSummarizer, MatchSummarizerCmpModel = MS.MatchSummarizer, MS.MatchSummarizerCmpModel

import sys
sys.path.append('..')
from Datasets.DataLoader import DataLoader

# loading the datasets
datasetLoader = DataLoader(datasetName='arxiv')

arxiv_test = datasetLoader.getData('../Datasets/', split='test')
datasetLoader.datasetName = 'pubmed'
pubmed_test = datasetLoader.getData('../Datasets/', split='test')

# pick only the first 1000 rows from the dataframes
arxiv_test = arxiv_test[:10]
pubmed_test = pubmed_test[:10]

# creating 'Gold Summary' column
def mapping(row):
    row['Gold Summary'] = ''.join(row['abstract_text'])
    return row

arxiv_test = arxiv_test.apply(mapping, axis=1)
pubmed_test = pubmed_test.apply(mapping, axis=1)

In [17]:
# loading the model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model2 = SentenceTransformer('ArtifactAI/arxiv-distilbert-base-v3-GenQ')

# generating summaries
def generateSummary(row):
    article = ''.join(row['article_text'])
    summarizer = MatchSummarizerCmpModel(article, model2, model2)
    summary = summarizer.generateSummary()
    row['Generated Summary'] = summary
    print(f"Generated summary for {row['article_id']}.")
    return row

start_time = time.time()
arxiv_test = arxiv_test.apply(generateSummary, axis=1)
print('Time taken for arxiv: ', (time.time() - start_time)/60, 'minutes.')

# start_time = time.process_time()
# pubmed_test = pubmed_test.apply(generateSummary, axis=1)
# print('Time taken for pubmed: ', (time.process_time() - start_time)/60, 'minutes.')

Generated summary for 1009.3123.
Generated summary for 1512.09139.
Generated summary for 0909.1602.
Generated summary for 1512.03812.
Generated summary for 1512.09024.
Generated summary for 0807.5065.
Generated summary for 0908.1812.
Generated summary for hep-ph0701277.
Generated summary for 1311.0649.
Generated summary for nlin0001046.
Time taken for arxiv:  4.441486644744873 minutes.


In [21]:
# evaluating the summaries
import Evaluation.evaluation as evaluation
import importlib
importlib.reload(evaluation)

rougeScores = evaluation.rougeScores
arxiv_test, rougeScoresArxiv = rougeScores(arxiv_test)
# pubmed_test, rougeScoresPubmed = rougeScores(pubmed_test)

# printing the results
print('arxiv')
print('rouge1: ', np.mean([ score.fmeasure for score in rougeScoresArxiv['rouge1'] ]))
print('rouge2: ', np.mean([ score.fmeasure for score in rougeScoresArxiv['rouge2'] ]))
print('rougeL: ', np.mean([ score.fmeasure for score in rougeScoresArxiv['rougeL'] ]))
print('bertScore: ', np.mean(arxiv_test['bertScore']))
# print('pubmed')
# print('rouge1: ', np.mean([ score.fmeasure for score in rougeScoresPubmed['rouge1'] ]))
# print('rouge2: ', np.mean([ score.fmeasure for score in rougeScoresPubmed['rouge2'] ]))
# print('rougeL: ', np.mean([ score.fmeasure for score in rougeScoresPubmed['rougeL'] ]))

# saving the results
arxiv_test.to_csv('arxiv_test_matchSum.csv')
# pubmed_test.to_csv('pubmed_test_matchSum.csv')

KeyboardInterrupt: 